In [8]:
#!pip install mysql
#!pip install mysql-connector-python-rf
#import warnings
#warnings.filterwarnings("ignore")

You are using pip version 9.0.1, however version 19.3.1 is available.
You should consider upgrading via the 'python -m pip install --upgrade pip' command.


In [1]:
from flask import Flask
from flask import url_for, render_template, request, redirect
import mysql.connector as mysql



In [291]:
con = mysql.connect(user = 'root', password = 'password', 
                    host = '127.0.0.1', database = 'db_proj', #database = 'task_manager'
                   auth_plugin='mysql_native_password')
cur = con.cursor(dictionary=True)

In [292]:
session = {}
session['username'] = None

In [293]:
app = Flask(__name__)

In [294]:
@app.route('/home', methods=['GET', 'POST'])
def index():
    if session['username'] == None:
        return render_template('home.html')
    else:
        tasks = []
        importance = []
        statuses = []
        ids = []
        base_string = "SELECT id, task, importance, status FROM (SELECT user_tasks.id_user_task as \'id\', tasks_list.task_name as \'task\', importance_list.importance_name as \'importance\',\
status_list.status_name as \'status\' FROM user_tasks, users_list, tasks_list, status_list, db_proj.importance_list \
WHERE users_list.username = '" + session['username'] + "' and users_list.id_user = user_tasks.id_user \
and tasks_list.id_task = user_tasks.id_task and status_list.id_status = user_tasks.id_status \
and importance_list.id_importance = user_tasks.id_importance) as t"
        if request.method == "POST":
            det = request.form
##            with open ('test.txt', 'a') as f:
##                f.write(det['importance1'])
            importance_list = []
            status_list = []
            if 'importance1' in det:
                importance_list.append(det['importance1'])
            if 'importance2' in det:
                importance_list.append(det['importance2'])
            if 'importance3' in det:
                importance_list.append(det['importance3'])
    
            if 'status1' in det:
                status_list.append(det['status1'])
            if 'status2' in det:
                status_list.append(det['status2'])
            if 'status3' in det:
                status_list.append(det['status3'])
           
            if not importance_list and not status_list:
                 
                cur.execute(base_string)
                result = cur.fetchall()
            else:
                base_string += ' WHERE ('
                if importance_list:
                    for i, element in enumerate(importance_list):
                        base_string = base_string + 'importance = \'' + element + '\''
                        if i != len(importance_list) - 1:
                            base_string += ' OR '
                        else:
                            base_string += ')'
                if status_list:
                    if importance_list:
                        base_string += ' AND ('
                    for i, element in enumerate(status_list):
                        base_string = base_string + 'status = \'' + element + '\''
                        if i != len(status_list) - 1:
                            base_string += ' OR '
                        else:
                            base_string += ')'
            cur.execute(base_string)  
            result = cur.fetchall()
        else:
            cur.execute(base_string)
            result = cur.fetchall()
        data = []
        for res in result:
            elem = [res['task'], res['importance'], res['status'], res['id']]
            data.append(elem)
#            ids.append(res['id'])
#            tasks.append(res['task'])
#            importance.append(res['importance'])
#            statuses.append(res['status'])
        return render_template('home_logged_in.html', data = data)

In [295]:
@app.route('/friends_tasks', methods=['GET', 'POST'])
def my_friends_tasks():
    tasks = []
    importance = []
    statuses = []
    ids = []
    
    # еще джойн (получаем список юзернеймов друзей юзера):
    cur.execute("SELECT username from users_list \
                JOIN (select id2 from users_list, user_relations \
                where id1 = id_user and username = '" + session['username'] + "') as f_id \
                ON id2 = id_user")  
    res = cur.fetchall()
    friends_list = [result['username'] for result in res]
    
    if friends_list == []: # если нет никаких друзей
        return render_template('no_friends.html')
    
    else: # если есть хотя бы один друг
        base_string = "SELECT username, task, importance, status FROM \
        (SELECT username, tasks_list.task_name as 'task', \
        importance_list.importance_name as 'importance', status_list.status_name as 'status' \
        FROM user_tasks, users_list, tasks_list, status_list, db_proj.importance_list \
        WHERE ("
        for id_f, f_username in enumerate(friends_list):
            base_string = base_string + "users_list.username = '" + f_username + "'"
            if id_f != len(friends_list) - 1:
                base_string += ' OR '
            else:
                base_string += ')'
        end_of_subquery = " and users_list.id_user = user_tasks.id_user \
        and tasks_list.id_task = user_tasks.id_task and status_list.id_status = user_tasks.id_status \
        and importance_list.id_importance = user_tasks.id_importance) as t"
        base_string += end_of_subquery
        
        if request.method == "POST":
            det = request.form
            importance_list = []
            status_list = []
            friend = det['friend']
        
            if 'importance1' in det:
                importance_list.append(det['importance1'])
            if 'importance2' in det:
                importance_list.append(det['importance2'])
            if 'importance3' in det:
                importance_list.append(det['importance3'])
    
            if 'status1' in det:
                status_list.append(det['status1'])
            if 'status2' in det:
                status_list.append(det['status2'])
            if 'status3' in det:
                status_list.append(det['status3'])
           
            if not importance_list and not status_list and friend == 'all':            
                cur.execute(base_string)
                result = cur.fetchall()
            else:
                base_string += ' WHERE ('
                if importance_list:
                    for i, element in enumerate(importance_list):
                        base_string = base_string + "importance = '" + element + "'"
                        if i != len(importance_list) - 1:
                            base_string += ' OR '
                        else:
                            base_string += ')'
                if status_list:
                    if importance_list:
                        base_string += ' AND ('
                    for i, element in enumerate(status_list):
                        base_string = base_string + "status = '" + element + "'"
                        if i != len(status_list) - 1:
                            base_string += ' OR '
                        else:
                            base_string += ')'
                if friend != 'all':
                    if importance_list or status_list:
                        base_string = base_string + " AND (username = '" + friend + "')"
                    else:
                        base_string = base_string + "username = '" + friend + "')"
                        
            cur.execute(base_string)  
            result = cur.fetchall()
        else:
            cur.execute(base_string)
            result = cur.fetchall()
        data = []
        for res in result:
            elem = [res['task'], res['importance'], res['status'], res['username']]
            data.append(elem)

        return render_template('friends_tasks.html', data = data, friends = friends_list)

In [296]:
@app.route('/signup', methods=['GET', 'POST'])
def sign_up():
    if request.method == "POST":
        details = request.form
        username = details['username']
        password = details['password']
        cur.execute("INSERT INTO users_list (username, password) VALUES (%s, %s)", (username, password))
        con.commit()
        return redirect('login')
    return render_template('sign_up.html')

In [297]:
@app.route('/login', methods=['GET', 'POST'])
def login():
    if request.method == "POST":
        details = request.form
        username = details['username']
        password = details['password']
        cur.execute("SELECT * FROM users_list")
        users = cur.fetchall()
        user_exists = 0
        for i in users:
            if i['username'] == username:
                user_exists = 1
                if i['password'] == password:
                    session['username'] = username
                    cur.execute("SELECT id_user FROM users_list WHERE username = '" + username + "'")
                    session['id'] = str(cur.fetchone()['id_user'])
                    return redirect('home')
                else:
                    return render_template('wrong_password.html')
        if user_exists == 1:
            return redirect ('home')
        else:
            return render_template('no_user.html')
    return render_template('login.html')

In [298]:
@app.route('/new_task', methods=['GET', 'POST'])
def new_task(): 
    if request.method == "POST":
        details = request.form
        cur.execute("SELECT id_user FROM users_list WHERE username = '" + session['username'] + "'")
        res = cur.fetchone()
        id_user = res['id_user']

##        cur.execute("SELECT id_task FROM tasks_list WHERE task_name = '" + details['tasks'] + "'")
##        res = cur.fetchone()
##       id_task = res['id_task']
        
        cur.execute("SELECT id_status FROM status_list WHERE status_name = '" + details['status'] + "'")
        res = cur.fetchone()
        id_status = res['id_status']
        
        cur.execute("SELECT id_importance FROM importance_list WHERE importance_name = '" + details['importance'] + "'")
        res = cur.fetchone()
        id_importance = res['id_importance']
        
        cur.execute("SELECT task_name, id_task FROM tasks_list")
        res = cur.fetchall()
        task_exists = 0
        for i in res:
            if i['task_name'] == details['tasks']:
                task_exists = 1
                id_task = i['id_task']
        if task_exists == 0:        
            cur.execute("INSERT INTO tasks_list (task_name) VALUES ('" + details['tasks'] + "')")
            cur.execute("SELECT id_task FROM tasks_list WHERE task_name = '" + details['tasks'] + "'")
            res = cur.fetchone()
            id_task = res['id_task']
        data = (id_user, id_task, id_status, id_importance)
        cur.execute("INSERT INTO user_tasks (id_user, id_task, id_status, id_importance) VALUES (%s, %s, %s, %s)", data)
        con.commit()
        return redirect('home')
    importance = []
    status = []
    cur.execute("SELECT importance_name FROM importance_list")
    res = cur.fetchall()
    for i in res:
        importance.append(i['importance_name'])
    cur.execute("SELECT status_name FROM status_list")
    res = cur.fetchall()
    for i in res:
        status.append(i['status_name'])
    return render_template('new_task.html', status = status, importance = importance)

In [299]:
@app.route('/update', methods=['GET', 'POST'])
def update_task():
    user_task_id = request.args.get('id') # должно быть в этом месте
    if request.method == "GET":
        # ура, джойн!:
        cur.execute("select task_name from tasks_list join user_tasks on \
        user_tasks.id_task = tasks_list.id_task and id_user_task = " + user_task_id)
        res = cur.fetchone()
        task = res['task_name']
    if request.method == "POST":
        details = request.form
        
        cur.execute("SELECT id_status FROM status_list WHERE status_name = '" + details['status'] + "'")
        res = cur.fetchone()
        id_status = res['id_status']
        
        cur.execute("SELECT id_importance FROM importance_list WHERE importance_name = '" + details['importance'] + "'")
        res = cur.fetchone()
        id_importance = res['id_importance']
        
        cur.execute("UPDATE user_tasks SET id_status = " + str(id_status) + ", id_importance = " + str(id_importance) + " \
        WHERE id_user_task = " + str(user_task_id))
        con.commit()
        return redirect('home')
    
    importance = []
    status = []
    cur.execute("SELECT importance_name FROM importance_list")
    res = cur.fetchall()
    for i in res:
        importance.append(i['importance_name'])
    cur.execute("SELECT status_name FROM status_list")
    res = cur.fetchall()
    for i in res:
        status.append(i['status_name'])
    return render_template('update_task.html', status = status, importance = importance, task = task)

In [300]:
@app.route('/delete', methods=['POST', 'GET'])
def delete():
    if request.args.get('id'):
        task_id = request.args.get('id')
        cur.execute("DELETE FROM user_tasks WHERE id_user_task = '" + task_id + "'")
        con.commit()
        return redirect('home')
    else:
        return('this page is meant to delete something')        

In [301]:
@app.route('/friends', methods = ['POST', 'GET'])
def friends():
    cur.execute("SELECT id2 from user_relations WHERE id1 = '" + session['id'] + "'")
    result = cur.fetchall()
    friends_ids = []
    requests = []
    for i in result:
        friends_ids.append(str(i['id2']))
    friends_usernames = []
    for i in friends_ids:
        cur.execute("SELECT username FROM users_list WHERE id_user = '" + i + "'")
        friends_usernames.append(cur.fetchone()['username'])
    
    cur.execute("SELECT friend_requests FROM users_list WHERE id_user = '" + session['id'] + "'")
    req_number = cur.fetchone()['friend_requests']
    if req_number > 0:
        cur.execute("SELECT id_from FROM friend_requests_list WHERE id_to = '" + session['id'] + "'")
        result = cur.fetchall()
        for i in result:
            cur.execute("SELECT id_user, username FROM users_list WHERE id_user = '" + str(i['id_from']) + "'")
            user_data = []
            res = cur.fetchone()
            user_data.append(res['id_user'])
            user_data.append(res['username'])
            requests.append(user_data)
    return render_template('friends.html', usernames = friends_usernames, requests = requests)

In [302]:
@app.route('/send_request', methods = ["POST", "GET"])
def send_request():
    if request.args.get('username'):
        username = request.args.get('username')
        cur.execute("SELECT id_user FROM users_list WHERE username = '" + username + "'")
        res = cur.fetchone()
        if res:
            id_friend = str(res['id_user'])
            cur.execute("SELECT friend_requests FROM users_list WHERE id_user = '" + id_friend + "'")
            req_number = cur.fetchone()['friend_requests']
            req_number = str(req_number + 1)
            cur.execute("UPDATE users_list SET friend_requests = '" + req_number + "' WHERE id_user = '" + id_friend + "'")
            cur.execute("INSERT INTO friend_requests_list (id_from, id_to) VALUES (%s, %s)", (session['id'], id_friend))
            con.commit()
            return redirect('/friends')
        else:
            return render_template('no_user.html')
    else:    
    
        return ('Не получил юзернейм.')

In [303]:
@app.route('/reply_to_request', methods = ["POST", "GET"])
def reply_to_request():
    if request.args.get('friend_req'):
        id_from = str(request.args.get('id_from'))
        if request.args.get('friend_req') == 'Принять':
            cur.execute("INSERT INTO user_relations (id1, id2) VALUES (%s, %s)", (session['id'], id_from))
            cur.execute("INSERT INTO user_relations (id1, id2) VALUES (%s, %s)", (id_from, session['id']))
        cur.execute("DELETE FROM friend_requests_list WHERE id_from = %s AND id_to = %s", (id_from, session['id']))
        cur.execute("SELECT friend_requests FROM users_list WHERE id_user = '" + session['id'] + "'")
        req_number = cur.fetchone()['friend_requests']
        req_number = str(req_number - 1)
        cur.execute("UPDATE users_list SET friend_requests = '" + req_number + "' WHERE id_user = '" + session['id'] + "'")
        con.commit()
        return redirect('/friends')
    else:
        return ('You should not be here')

In [304]:
if __name__ == '__main__':
    app.run(debug=False)

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
127.0.0.1 - - [26/Dec/2019 14:52:00] "POST /login HTTP/1.1" 302 -
127.0.0.1 - - [26/Dec/2019 14:52:00] "GET /home HTTP/1.1" 200 -
127.0.0.1 - - [26/Dec/2019 14:52:02] "GET /friends HTTP/1.1" 200 -
127.0.0.1 - - [26/Dec/2019 14:52:06] "GET /new_task HTTP/1.1" 200 -
127.0.0.1 - - [26/Dec/2019 14:52:07] "GET /home HTTP/1.1" 200 -
127.0.0.1 - - [26/Dec/2019 14:52:11] "GET /friends HTTP/1.1" 200 -
127.0.0.1 - - [26/Dec/2019 14:52:14] "GET /send_request?username=ejouny HTTP/1.1" 302 -
127.0.0.1 - - [26/Dec/2019 14:52:14] "GET /friends HTTP/1.1" 200 -
127.0.0.1 - - [26/Dec/2019 14:52:16] "GET /login HTTP/1.1" 200 -
127.0.0.1 - - [26/Dec/2019 14:52:25] "POST /login HTTP/1.1" 302 -
127.0.0.1 - - [26/Dec/2019 14:52:25] "GET /home HTTP/1.1" 200 -
127.0.0.1 - - [26/Dec/2019 14:52:27] "GET /friends HTTP/1.1" 200 -
127.0.0.1 - - [26/Dec/2019 14:52:32] "GET /reply_to_request?id_from=5&friend_req=Принять HTTP/1.1" 302 -
127.0.0.1 - - [26/Dec/

Ниже фигня

In [306]:
cur.execute("SELECT * FROM users_list")
result = cur.fetchall()
result

[{'friend_requests': 0,
  'id_user': 1,
  'password': 'йцукен',
  'username': 'fishowl'},
 {'friend_requests': 0,
  'id_user': 2,
  'password': 'фывапро',
  'username': 'mcqueen'},
 {'friend_requests': 0,
  'id_user': 3,
  'password': 'янееммясо',
  'username': 'ejouny'},
 {'friend_requests': 0,
  'id_user': 4,
  'password': 'галилеофигаро',
  'username': 'mashasu'},
 {'friend_requests': 0,
  'id_user': 5,
  'password': 'prince',
  'username': 'cinderella'}]

In [196]:
cur.execute("SELECT * FROM friend_requests_list")
result = cur.fetchall()
result

[{'id_from': 1, 'id_request': 1, 'id_to': 3}]

In [236]:
cur.execute("UPDATE users_list SET friend_requests = 0")

In [179]:
cur.execute("DELETE FROM friend_requests_list WHERE id_request > 1")

In [237]:
con.commit()

In [76]:
con.close()

In [290]:
result = cur.fetchall()
result

[{'friend_requests': 0}]